# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [14]:
from openai import OpenAI

# client = OpenAI()
client = OpenAI(base_url="https://api.xiaoai.plus/v1", api_key="uKAcWnrmqyOFdqNd625a799a054843638fC10596BaA014E4")

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图展示了一条木制的小径伸展在一片繁盛的绿色草地上。小径两侧的草地郁郁葱葱，远处可以看到一些树木和灌木。天空是清晰的，有着漂亮的蓝色和几朵散布的白云。这种景色赋予这幅画一种宁静和自然的感觉，适合散步或沉思。整体色彩鲜明，自然光线使得这个场景看起来生动而引人入胜。', role='assistant', function_call=None, tool_calls=None))


In [15]:
response.choices[0].message.content

'这幅图展示了一条木制的小径伸展在一片繁盛的绿色草地上。小径两侧的草地郁郁葱葱，远处可以看到一些树木和灌木。天空是清晰的，有着漂亮的蓝色和几朵散布的白云。这种景色赋予这幅画一种宁静和自然的感觉，适合散步或沉思。整体色彩鲜明，自然光线使得这个场景看起来生动而引人入胜。'

### 封装成一个函数 query_image_description

In [5]:
def query_image_description(url, prompt="介绍下这幅图?"):
    # client = OpenAI()  # 初始化 OpenAI 客户端
    client = OpenAI(base_url="https://api.xiaoai.plus/v1", api_key="uKAcWnrmqyOFdqNd625a799a054843638fC10596BaA014E4")
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [19]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图是一个幽默搞笑的表现，展示了两只经过夸张修改的狗的形象，一只显得格外强壮，另一只则显得更加普通和温和。左侧的狗被描绘成一个拥有健壮肌肉的体格，头部仍保持狗的特征，给人一种超现实和荒谬的感觉。右侧的狗则坐着，显得更加自然和平常。

图中的文字则为这两种极端形态提供了一个幽默的对比，左侧为“16岁的我，工作后的我”，意味着年轻时的强健和充满活力，工作后则变得普通和压力重重。右侧则描述了工作的消极影响和压力，如“脱发很严重”和“朋友都变味道”。

整体来看，这幅图通过夸张和幽默的对比展示了人


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [7]:
from openai import OpenAI
import base64
import requests
import json

# client = OpenAI()  # 初始化 OpenAI 客户端
client = OpenAI(base_url="https://api.xiaoai.plus/v1", api_key="uKAcWnrmqyOFdqNd625a799a054843638fC10596BaA014E4")

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [32]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

这张图展示了1980年至2020年期间，美国（蓝色线）、中国（红色线）、日本（紫色线）和德国（绿色线）四个国家国内生产总值（GDP）的比较。图中的纵轴表示GDP量（万亿美元），横轴表示年份。

从图中可以看出：
- 美国的GDP始终保持在四个国家中最高，且呈逐年增长趋势，到2020年达到约21万亿美元。
- 中国的GDP增长非常迅速，特别是从2000年开始，增长率显著提高，到2020年接近15万亿美元，是1980年时的数十倍。
- 日本的GDP在1995年前后达到顶峰，之后有所波动，但整体保持在4至6万亿美元之间。
- 德国的GDP增长相对平稳，到2020年约为4万亿美元。

总的来说，该图表清晰地展示了这四个经济体在过去40年中GDP的变化趋势，特别是突出了中国的快速增长。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [10]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

这张图片显示的是一本笔记本上手写的文字，内容主要涉及到自然语言处理（NLP）中的训练技术，如Prompt Tuning和LoRA技术。

1. **Prompt Tuning（简要模型调整）**： 提到了使用Prompt Tuning来调整一个小型的Transformer模型。此处解释了输入X（由个别输入X1, X2, ..., Xn组成）。每个输入首先通过一个Embedding过程转换，然后通过Token变换。输出Y是通过矩阵W与转换后的输入X'之间的乘法得出。

2. **Prefix Tuning：** 这部分说明了Prefix Tuning的过程，其中添加了前缀权重W_p到原始权重W_j中，得到新的权重W'用于生成输出Y。

3. **LoRA调整技术**： 这部分涉及Linear Re-parameterization（线性重新参数化），通过调整矩阵ΔW（通过两个矩阵A和B的乘积表示）来修改权重W。这是一种节省参数调整的方法，使原有模型的W变为W+ΔW，这里也涉及到了一些矩阵运算和优化策略。

其中还提到了两个案例分析的存储需求：“LLAMA”需要65GB，而经过LoRA调整的“QLLoRA”仅需要48GB。

这些笔记对于理解NLP中一些先进的模型调整技术十分有用，尤其对于需要在资源受限的环境下部署NLP模型的研究人员或实践者。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [8]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

NameError: name 'content' is not defined

![](./images/handwriting_1.jpg)

In [24]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这张图片展示了一本笔记，内容涉及深度学习中的一些方法和研究，尤其是与变换器（Transformers）和微调（Fine-tuning）相关的主题。以下是一些主要内容的总结：

1. **IF Transformers**：讨论变换器模型的基准和数据。
   
2. **PEFT (Parameter-Efficient Fine-Tuning)**：介绍了几种微调方法和最新的进展，包括：
   - 各种技术如 Adapter、Prefix Tuning、Prompt Tuning 等。
   - 具体的方法和年份（例如，2019 年的 Adapter，2021 年的 Prefix Tuning 和 Prompt Tuning）。

3. **Multi-modality Instruction**：提到不同类型的多模态指令模型，及其与大语言模型（LLMs）的关系。

4. **Lora**：讨论包括 Lora、QLora 和 Adalora 的不同方法。

5. **Prefix-tuning & Adapters**：分析了在大语言模型中使用前缀微调和适配器的技术，提出了关于如何精细调整模型的一些策略。

6. **参考文献**：提到了一些研究和文献，可能是作者在深入研究时参考的材料。

整本笔记似乎在整理和归纳深度学习特别是自然语言处理领域的前沿技术和方法。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。

In [23]:
content = query_base64_image_description("./images/chat-4v.jpg")
display(Markdown(content))

这段文字介绍了珠海市的基本信息。珠海市位于广东省，是一个地级市，属于经济和文化发展的重要城市。文字中提到：

- 珠海市是广东省的副中心城市。
- 其面积和人口数据，包括总面积和常住人口。
- 还有涉及到珠海市的行政区划，包括香洲区、斗门区和金湾区。

总体来说，这段内容概括了珠海市的地理位置、行政区划和人口信息。

In [27]:
content = query_base64_image_description("./images/chat-4v.jpg", prompt="给我用markdown的格式打印出图片的内容， 然后再根据内容回答是哪个城市，它的占地面积多大")
display(Markdown(content))

```markdown
珠海市  
广东省地级市、省域副中心城市，型大城市，位于广东省南部，珠江口海域西缘，临南海，陆地面积1725平方千米，领海基线以内海域面积9348平方千米。下辖香洲区、斗门区、金湾区三个行政区。截止2023年末，珠海市（含横琴粤澳深度合作区）常住人口约为24.91万人。
```

根据内容，这个城市是 **珠海市**。它的占地面积为 **1725 平方公里**。